In [13]:
import pandas as pd
import numpy as np
import sqlite3

In [14]:
pd.read_csv('Badges.csv')

,Class,Date,Id,Name,TagBased,UserId
0,3,2010-08-25T19:26:49.423,1,Autobiographer,False,2
1,3,2010-08-25T19:26:49.440,2,Autobiographer,False,3
2,3,2010-08-25T19:26:49.457,3,Autobiographer,False,4
3,3,2010-08-25T19:41:49.283,4,Autobiographer,False,6
4,3,2010-08-25T19:41:49.283,5,Autobiographer,False,7
...,...,...,...,...,...,...
66584,3,2019-09-01T01:20:05.347,80315,Popular Question,False,33825
66585,3,2019-09-01T02:25:19.667,80316,Student,False,44925
66586,3,2019-09-01T02:35:09.420,80317,Student,False,38048
66587,2,2019-09-01T02:45:11.520,80318,Yearling,False,1502


In [4]:
names = (
    'Badges',
    'PostLinks',
    'Tags',
    'Votes',
    'Comments',
    'Posts',
    'Users'
)

for name in names:
    print(name)
    print(
        pd.read_csv(f"{name}.csv").head()
    )
    print('-'*10)

Badges
   Class                     Date  Id            Name  TagBased  UserId
0      3  2010-08-25T19:26:49.423   1  Autobiographer     False       2
1      3  2010-08-25T19:26:49.440   2  Autobiographer     False       3
2      3  2010-08-25T19:26:49.457   3  Autobiographer     False       4
3      3  2010-08-25T19:41:49.283   4  Autobiographer     False       6
4      3  2010-08-25T19:41:49.283   5  Autobiographer     False       7
----------
PostLinks
              CreationDate  Id  LinkTypeId  PostId  RelatedPostId
0  2010-08-26T20:33:50.797  31           1     262            261
1  2010-08-29T21:51:09.093  47           1     401            244
2  2010-08-30T01:57:19.137  48           1     132              3
3  2010-08-31T19:13:55.077  65           1     367             27
4  2010-08-31T19:13:55.077  66           1     367            246
----------
Tags
   Count  ExcerptPostId  Id    TagName  WikiPostId
0     21            NaN   1   schrader         NaN
1     44            NaN   

In [12]:
conn = sqlite3.connect('pandas_exercises')

for name in names:
    print(name)
    ### Wczytywanie pliku csv do zmiennej df
    df = pd.read_csv(f"{name}.csv")
    ### Zapisywanie do bazy danych
    df.to_sql(name, conn, if_exists='replace')


Badges
PostLinks
Tags
Votes
Comments
Posts
Users


In [6]:
dfs = {name: pd.read_csv(f"{name}.csv")
      for name in names}

In [7]:
query1 = '''
-----Zadanie 1
SELECT PostId, COUNT(*) AS UpVotes
FROM Votes
WHERE VoteTypeID=2
GROUP BY PostId
'''

In [8]:
pd.read_sql_query(query1, con = conn)

,PostId,UpVotes
0,1,42
1,2,13
2,3,10
3,4,16
4,5,31
...,...,...
41776,63933,2
41777,63934,1
41778,63939,1
41779,63940,1


In [9]:
# query0 =  '''
# SELECT *
# FROM Votes
# WHERE Votes_IS=2'''

In [18]:
pd.read_sql_query(query0, con = conn)

DatabaseError: Execution failed on sql '
SELECT *
FROM Votes
WHERE Votes_IS=2': no such column: Votes_IS

In [ ]:
votes_df = dfs['Votes']
votes_df = votes_df[votes_df['VoteTypeId']==2]
votes_df

In [ ]:
query1 = '''
-----Zadanie 1
SELECT PostId, COUNT(*) AS UpVotes
FROM Votes
WHERE VoteTypeID=2
GROUP BY PostId
'''

In [ ]:

votes_df = dfs['Votes']
votes_df = votes_df[votes_df['VoteTypeId']==2]
votes_df = votes_df.groupby('PostId')[['VoteTypeId']].count()
votes_df

In [ ]:
query2 = """
--- Zadanie 2
SELECT Title, Score, ViewCount, FavoriteCount
FROM Posts
WHERE PostTypeId=1 AND FavoriteCount >= 25 AND ViewCount>=10000
"""

In [ ]:
posts_df = dfs['Posts']
posts_df = posts_df[(posts_df['PostTypeId'] == 1) & (posts_df['FavoriteCount'] >= 25) & (posts_df['ViewCount'] >= 10000)]
posts_df = posts_df[['Title', 'Score', 'ViewCount', 'FavoriteCount']]
posts_df

In [ ]:
query3 = """
--- Zadanie 3
SELECT Tags.TagName, Tags.Count, Posts.OwnerUserId,
Users.Location, Users.DisplayName
FROM Tags
JOIN Posts ON Posts.Id=Tags.WikiPostId
JOIN Users ON Users.AccountId=Posts.OwnerUserId
WHERE OwnerUserId != -1
ORDER BY Count DESC
"""

In [ ]:
tags_df = dfs['Tags']
posts_df = dfs['Posts']
users_df = dfs['Users']
posts_df = posts_df[(posts_df['OwnerUserId'])!= -1]
tags_post_df = pd.merge(tags_df, posts_df, left_on = 'WikiPostId', right_on ='Id')
tags_post_user_df = pd.merge(tags_post_df, users_df, left_on = 'OwnerUserId', right_on ='AccountId')
tags_post_user_df = tags_post_user_df[['TagName', 'Count', 'OwnerUserId', 'Location', 'DisplayName']]
tags_post_user_df.sort_values('Count', ascending = False)

In [ ]:
# query4 = """
# SELECT Posts.Title, RelatedTab.NumLinks FROM
# (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks FROM PostLinks GROUP BY RelatedPostId) AS RelatedTab
# JOIN Posts ON RelatedTab.PostId=Posts.Id WHERE Posts.PostTypeId=1
# ORDER BY NumLinks DESC
# """

In [33]:
postslinks_df = dfs['PostLinks']
postslinks_df = postslinks_df.groupby('RelatedPostId')[['CreationDate']].count().reset_index()
#reset index - zmienia indeks w tabele

# planes_df_copy = planes_df[['engine', 'type', 'tailn
postslinks_df.columns = ['PostId', 'NumLinks']
postslinks_df

,PostId,NumLinks
0,1,5
1,3,6
2,4,1
3,5,6
4,6,1
...,...,...
2228,63118,1
2229,63252,2
2230,63517,1
2231,63599,1


In [48]:
posts_df = dfs['Posts']
posts_df = posts_df[posts_df['PostTypeId'] ==1]
postlinks_join = postslinks_df.merge(posts_df, left_on = 'PostId', right_on='Id')
postlinks_join = postlinks_join[['Title', 'NumLinks']]
postlinks_join = postlinks_join.sort_values(['NumLinks'], ascending= False)
postlinks_join
# sort_values(['year','seats'], ascending= [True, False])

,Title,NumLinks
42,Terminology index - a list of bike part names ...,156
2064,Why shouldn't I care what model/make/year my b...,154
1995,How can I tell what year my bike was made?,100
2077,What is the maximum or minimum tire width I ca...,42
627,Front suspension or full suspension mountain bike,37
...,...,...
406,I am about to buy a new bicycle - what should ...,1
849,Disassembly and Shipping of bike,1
1340,Rotating two chains on one cassette,1
850,I was in a collision the other day. What shoul...,1


In [49]:
# --- Zadanie 5
# SELECT UpVotesTab.*, Posts.Title FROM (
# SELECT PostId, COUNT(*) AS UpVotes FROM Votes WHERE VoteTypeId=2 GROUP BY PostId ) AS UpVotesTab
# JOIN Posts ON UpVotesTab.PostId=Posts.Id WHERE Posts.PostTypeId=1
# ORDER BY UpVotesTab.UpVotes DESC


In [62]:
posts_df = dfs['Posts']
votes_df = dfs['Votes']
votes_df = votes_df[votes_df['VoteTypeId'] ==2]
votes_df = votes_df.groupby('PostId')[['CreationDate']].count().reset_index()
votes_df.columns = ['PostId','UpVotes']
votes_df
# postslinks_df.columns = ['PostId', 'NumLinks']

posts_df = posts_df[posts_df['PostTypeId']==1]
posts_df

# tags_post_user_df = pd.merge(tags_post_df, users_df, left_on = 'OwnerUserId', right_on ='AccountId')

join_tab = pd.merge(votes_df, posts_df, left_on = 'PostId', right_on = 'Id' )
join_tab

# UpVotesTab.*, Posts.Title
join_tab = join_tab[['Title','UpVotes','PostId']]
join_tab


# sort_values(['NumLinks'], ascending= False) .. UpVotesTab.UpVotes DESC
join_tab = join_tab.sort_values(['UpVotes'], ascending= False)
join_tab



SyntaxError: EOL while scanning string literal (Temp/ipykernel_1556/1213334168.py, line 14)

In [65]:
# --- Zadanie 6
# SELECT UpVotesTab.PostId, UpVotesTab.UpVotes, IFNULL(DownVotesTab.DownVotes, 0) AS DownVot FROM
# (
# SELECT PostId, COUNT(*) AS UpVotes FROM Votes
# WHERE VoteTypeId=2 GROUP BY PostId ) AS UpVotesTab
# LEFT JOIN
# (
# SELECT PostId, COUNT(*) AS DownVotes FROM Votes WHERE VoteTypeId=3 GROUP BY PostId
# ) AS DownVotesTab
# ON UpVotesTab.PostId=DownVotesTab.Post
votes_df = dfs['Votes']

UpVotesTab_df = votes_df[votes_df['VoteTypeId'] == 2].groupby('PostId')[['Id']].count().reset_index()
UpVotesTab_df.columns = ['PostId', 'UpVotes']
DownVotesTab_df = votes_df[votes_df['VoteTypeId'] == 3].groupby('PostId')[['Id']].count().reset_index()
DownVotesTab_df.columns = ['PostId', 'DownVotes']

UpDownVotes_df = pd.merge(UpVotesTab_df, DownVotesTab_df, on='PostId', how='left')
UpDownVotes_df = UpDownVotes_df.fillna(0)

UpDownVotes_df

,PostId,UpVotes,DownVotes
0,1,42,0.0
1,2,13,0.0
2,3,10,0.0
3,4,16,0.0
4,5,31,0.0
...,...,...,...
41776,63933,2,0.0
41777,63934,1,0.0
41778,63939,1,0.0
41779,63940,1,0.0


In [67]:
# --- Zadanie 7
votes_df = dfs['Votes']

UpVotesTab_df = votes_df[votes_df['VoteTypeId'] == 2].groupby('PostId')[['Id']].count().reset_index()
UpVotesTab_df.columns = ['PostId', 'UpVotes']
DownVotesTab_df = votes_df[votes_df['VoteTypeId'] == 3].groupby('PostId')[['Id']].count().reset_index()
DownVotesTab_df.columns = ['PostId', 'DownVotes']

UpDownVotes_df = pd.merge(UpVotesTab_df, DownVotesTab_df, on='PostId', how='outer')
UpDownVotes_df = UpDownVotes_df.fillna(0)

UpDownVotes_df['Votes'] = UpDownVotes_df['UpVotes'] - UpDownVotes_df['DownVotes']

UpDownVotes_df.sort_values('Votes', ascending = False)

,PostId,UpVotes,DownVotes,Votes
1887,1980,174.0,3.0,171.0
236,244,167.0,2.0,165.0
1888,1981,119.0,0.0,119.0
7189,8233,110.0,1.0,109.0
297,308,103.0,2.0,101.0
...,...,...,...,...
44269,46084,0.0,10.0,-10.0
45195,57614,0.0,10.0,-10.0
43309,34510,0.0,11.0,-11.0
37588,57313,2.0,13.0,-11.0
